In [ ]:
print("모든 필수 라이브러리를 최신 버전으로 설치")
!pip install -q -U transformers accelerate peft trl datasets pandas bitsandbytes

In [12]:
# --- 라이브러리 임포트 및 환경 설정 ---
print("\n 라이브러리 불러오기")

import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
import numpy as np
from huggingface_hub import notebook_login
from google.colab import drive
import os

os.environ["WANDB_DISABLED"] = "true" # Weights & Biases 로깅 기능 비활성화

drive.mount('/content/drive') # 드라이브 연결
notebook_login() # Hugging Face Hub 로그인

# --- 데이터 로드 및 전처리 ---
file_path = '/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/discharge.csv'
n_rows_to_read = 2000 # 상위 2000개의 행만 읽음
df = pd.read_csv(file_path, usecols=['subject_id', 'text'], nrows=n_rows_to_read)
df.dropna(subset=['text'], inplace=True)  # "text" 컬럼이 NaN일 경우, 해당 행 제거

unique_user_ids = df['subject_id'].unique() # ID 목록을 8:1:1 비율로 분할할 인덱스를 계산
np.random.shuffle(unique_user_ids)
train_split = int(len(unique_user_ids) * 0.8)
dev_split = int(len(unique_user_ids) * 0.9)
train_ids, dev_ids, test_ids = np.split(unique_user_ids, [train_split, dev_split])
train_df = df[df['subject_id'].isin(train_ids)]
dev_df = df[df['subject_id'].isin(dev_ids)]

# Hugging Face의 datasets 라이브러리가 사용하기 좋은 형태로 변환
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(dev_df)
})
print("데이터 분할 완료")

# SFTTrainer가 학습할 최종 텍스트 형태(프롬프트)를 만드는 함수를 정의]
def create_prompt(example):
    # 'example'은 데이터 한 개(row)
    # 'text' 컬럼의 내용을 가져와 프롬프트 형식으로 만듦
    return {
        'text': f"""### Instruction:
Summarize the key points from the following medical discharge summary.

### Input:
{example['text']}

### Response:
"""
    }

# .map() 함수를 사용하여 데이터셋의 모든 행에 create_prompt 함수를 미리 적용
# SFTTrainer는 기본적으로 text라는 이름의 컬럼을 찾기 때문에, 이 과정을 통해 원본 text 컬럼이 학습에 적합한 프롬프트가 담긴 새로운 text 컬럼으로 대체
column_names = raw_datasets['train'].column_names
formatted_datasets = raw_datasets.map(create_prompt, remove_columns=column_names)
print("\n데이터 포맷팅 완료")


# --- 모델 및 QLoRA 설정 ---
print("\n모델과 토크나이저를 로드")
model_name = "Qwen/Qwen2-7B-Instruct"
# 4비트 양자화(Quantization) 설정을 정의 (QLoRA의 'Q' 부분)
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
# LoRA(Low-Rank Adaptation) 어댑터 설정을 정의 (QLoRA의 'LoRA' 부분)
peft_config = LoraConfig(lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"])

# 위에서 정의한 양자화 설정을 적용하여 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model.config.use_cache = False
# 모델에 맞는 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("모델 설정 완료")


# --- 학습 실행 ---
print("\n파인튜닝 시작")
training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/MyModels/Qwen2_Finetune_Results", # 체크포인트 등 결과물이 저장될 경로
    num_train_epochs=1,                # 전체 데이터셋을 1번 학습
    per_device_train_batch_size=1,     # 한 번에 GPU에 올릴 데이터 샘플 수
    gradient_accumulation_steps=4,     # 4번의 스텝마다 그래디언트를 업데이트 (배치 사이즈 1*4=4 효과)
    optim="paged_adamw_32bit",         # 메모리 효율적인 AdamW 옵티마이저
    logging_steps=10,                  # 10 스텝마다 로그(loss 등) 출력
    learning_rate=2e-4,                # 학습률
    bf16=True,                         # A100/L4 GPU에서 성능을 높여주는 bfloat16 활성화
    max_grad_norm=0.3,                 # 그래디언트 클리핑 값
    warmup_ratio=0.03,                 # 학습 초반에 학습률을 서서히 증가시키는 구간 비율
    lr_scheduler_type="constant",      # 학습률을 일정하게 유지
)

# SFTTrainer에 불필요한 인자를 모두 제거하고,
# 미리 전처리된 데이터셋만 전달하여 가장 안정적으로 동작하도록 함
trainer = SFTTrainer(
    model=model,                       # 학습할 모델
    args=training_arguments,           # 학습 인자
    train_dataset=formatted_datasets["train"], # 훈련 데이터셋 (이미 'text' 컬럼만 있음)
    eval_dataset=formatted_datasets["validation"], # 평가 데이터셋
    peft_config=peft_config,           # LoRA 설정
)

trainer.train()

# --- 모델 저장 ---
print("\n학습된 모델을 저장")
# 학습된 LoRA 어댑터(튜닝 칩)만 지정된 경로에 저장
tuned_model_path = "/content/drive/MyDrive/MyModels/Qwen2_Finetuned_Adapter"
trainer.model.save_pretrained(tuned_model_path)
print(f"모델 어댑터가 '{tuned_model_path}'에 저장됨")


 라이브러리 불러오기
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


데이터 분할 완료


Map:   0%|          | 0/1606 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]


데이터 포맷팅 완료

모델과 토크나이저를 로드


...

Truncating eval dataset:   0%|          | 0/202 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.312600
20,2.022900
30,1.810700
40,1.791500
50,1.756600
60,1.758200
70,1.631600
80,1.715800
90,1.771300
100,1.663100


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



학습된 모델을 저장
모델 어댑터가 '/content/drive/MyDrive/MyModels/Qwen2_Finetuned_Adapter'에 저장됨


In [14]:
# --- 라이브러리 임포트 ---
import torch
# PEFT(LoRA)로 파인튜닝된 모델을 쉽게 불러오기 위한 클래스
from peft import AutoPeftModelForCausalLM
# 텍스트를 토큰으로 변환하는 토크나이저를 불러오기 위한 클래스
from transformers import AutoTokenizer

# --- 경로 설정 ---
# 구글 드라이브에 저장해 둔, 학습이 완료된 'LoRA 어댑터'의 경로
tuned_model_path = "/content/drive/MyDrive/MyModels/Qwen2_Finetuned_Adapter"
# 토크나이저는 파인튜닝 과정에서 변경되지 않았으므로,
# 원본 모델이 있던 허깅페이스 경로를 그대로 사용
base_model_name = "Qwen/Qwen2-7B-Instruct"

print("파인튜닝된 모델과 원본 토크나이저를 로드")

# --- 모델과 토크나이저 불러오기 ---
# AutoPeftModelForCausalLM.from_pretrained()는 두 가지 일을 동시에 수행:
#   1. 'base_model_name'에 해당하는 원본 Qwen2 모델을 허깅페이스에서 다운로드/로드
#   2. 'tuned_model_path'에 저장된 LoRA 어댑터(튜닝 칩)를 그 위에 합침
model = AutoPeftModelForCausalLM.from_pretrained(
    tuned_model_path,
    device_map="auto",          # 사용 가능한 GPU에 모델을 자동으로 할당
    torch_dtype=torch.bfloat16  # bfloat16 데이터 타입을 사용하여 메모리 효율성과 속도 향상
)

# 토크나이저는 어댑터 폴더가 아닌, 원본 모델의 허깅페이스 경로에서 불러옴
# 어댑터 폴더에는 토크나이저 정보가 없기 때문
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("모델 로딩 완료")

# --- 3. 테스트용 데이터(프롬프트) 준비 ---
# 모델에게 요약시킬 새로운 의료 기록 텍스트
test_summary = """
Dear Ms. ___,
It was a pleasure taking care of you! You came to us with
stomach pain and worsening distension. While you were here we
did a paracentesis to remove 1.5L of fluid from your belly. We
also placed you on you 40 mg of Lasix and 50 mg of Aldactone to
help you urinate the excess fluid still in your belly. As we
discussed, everyone has a different dose of lasix required to
make them urinate and it's likely that you weren't taking a high
enough dose. Please take these medications daily to keep excess
fluid off and eat a low salt diet. You will follow up with Dr.
___ in liver clinic and from there have your colonoscopy
and EGD scheduled. Of course, we are always here if you need us.
We wish you all the best!
Your ___ Team.
"""

# 모델이 학습했을 때 사용했던 프롬프트 양식을 "반드시" 똑같이 사용해야 함
prompt = f"""### Instruction:
Summarize the key points from the following medical discharge summary.

### Input:
{test_summary}

### Response:
"""

# --- 추론(Inference) 실행 ---
print("\n추론 시작")

# 1. 텍스트 프롬프트를 토크나이저를 사용해 모델이 이해할 수 있는 숫자(ID)의 배열로 변환
# 2. return_tensors="pt": 결과를 PyTorch 텐서 형태로 받음
# 3. .cuda(): 변환된 텐서를 GPU 메모리로 보냄
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

# model.generate() 함수를 호출하여 텍스트 생성을 시작합니다.
outputs = model.generate(
    input_ids=input_ids,          # 입력 텍스트
    max_new_tokens=100,           # 최대 몇 개의 토큰(단어)을 새로 생성할지 지정
    do_sample=True,               # 샘플링 방식을 사용하여 좀 더 창의적인 텍스트 생성
    top_p=0.9,                    # 확률이 높은 상위 90%의 단어들 중에서만 샘플링
    temperature=0.7               # 값이 낮을수록 예측 가능한 답변, 높을수록 다양한 답변 생성 (0.7은 약간의 창의성 부여)
)

# --- 결과 후처리 및 출력 ---
# 생성된 숫자(토큰) 배열을 다시 사람이 읽을 수 있는 텍스트로 변환(디코딩)
# skip_special_tokens=True: <|endoftext|> 같은 특수 토큰은 제외하고 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 전체 생성된 텍스트에서 우리가 입력한 프롬프트 부분을 제외하고,
# 순수하게 모델이 생성한 'Response' 부분만 추출하여 깔끔하게 만듦
response_text = result.split("### Response:")[1].strip()

# 최종 결과를 출력
print("\n" + "="*50)
print("파인튜닝된 모델의 요약 결과:")
print(response_text)
print("="*50)

파인튜닝된 모델과 원본 토크나이저를 로드


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

모델 로딩 완료

추론 시작

파인튜닝된 모델의 요약 결과:
Here is a summary of the key points from the medical discharge summary:

- The patient was admitted to the hospital due to stomach pain and worsening distension.
- A paracentesis was performed, removing 1.5 liters of fluid from the patient's abdomen.
- The patient was given 40 mg of Lasix and 50 mg of Aldactone to help reduce excess fluid in the abdomen.
- It was explained that everyone may require a different dose of Lasix, and the
